In [20]:
# 회귀
# 출력의 개수를 1개로
# 손실함수를 MSE나 기타 등등..
# 데이터셋 과 데이터 로드를 커스텀하게 정의해서 사용
# 나머지는 동일한 패턴으로 ... 학습 평가

In [21]:
# from sklearn.datasets import load_boston

In [26]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

class BostonDataSet(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1,1)
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\sally\AppData\Local\Temp\ipykernel_12640\362668587.py:7: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


In [28]:
import torch
X_dataset = BostonDataSet(data, target)  # 데이터를 X, y 를 한쌍으로 묶어
X_train_loader = DataLoader(X_dataset ,batch_size=32,shuffle=True)

AttributeError: partially initialized module 'torch' from 'c:\Users\sally\SKNPYWORKSPACE\streamlit\Lib\site-packages\torch\__init__.py' has no attribute 'nn' (most likely due to a circular import)

In [ ]:
# 회귀 모델 정의
import torch.nn as nn
class BostonRegression(nn.Module):
    def __init__(self, input_dim):
        super(BostonRegression,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
from torch.optim import Adam
model = BostonRegression(data.shape[1])
criterion = nn.MSELoss()
optim = Adam(model.parameters(), lr = 1e-3)

In [ ]:
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
epochs = 500
# 학습루프
for epoch in range(epochs):
    tqdm_obj = tqdm(X_train_loader,desc=f'epoch : {epoch+1}/{epochs}')
    for data, label in tqdm_obj:
        optim.zero_grad()
        preds = model(data.to(device))
        loss = criterion(preds, label.to(device))
        loss.backward()
        optim.step()

        tqdm_obj.set_postfix(loss=loss.item())
        

torch.save(model.state_dict(), 'bostonRegression.pth') 